In [ ]:
!pip install d2l

## import module

In [ ]:
import numpy as np
import tensorflow as tf
from d2l import tensorflow as d2l

## generate datasets

In [ ]:
true_w = tf.constant([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

## reading datasets

In [ ]:
def load_array(data_arrays, batch_size, is_train=True):  #@save"""Construct a TensorFlow data iterator."""
    dataset = tf.data.Dataset.from_tensor_slices(data_arrays)
    if is_train:
        dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(batch_size)
    return dataset

batch_size = 10
data_iter = load_array((features, labels), batch_size)

we use iter to construct a Python iterator and use next to obtain the first item from the iterator.

In [ ]:
next(iter(data_iter))

(<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
 array([[-0.7855348 ,  0.9105857 ],
        [-0.79782057,  0.6912928 ],
        [ 0.37471572, -2.0143378 ],
        [-0.15483612, -0.19595227],
        [-0.21722475,  0.8540268 ],
        [ 1.7054065 ,  0.5598915 ],
        [-0.7505607 , -0.02211357],
        [ 0.7265861 , -1.1477206 ],
        [-0.583126  , -0.6871914 ],
        [ 0.9766279 , -1.2944337 ]], dtype=float32)>,
 <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[-0.45619205],
        [ 0.2706712 ],
        [11.798034  ],
        [ 4.5473676 ],
        [ 0.85687184],
        [ 5.706851  ],
        [ 2.7971098 ],
        [ 9.576376  ],
        [ 5.3657393 ],
        [10.568655  ]], dtype=float32)>)

## defining the model

In [ ]:
# `keras` is the high-level API for TensorFlow
net = tf.keras.Sequential()
net.add(tf.keras.layers.Dense(1))

## initializing model parameters

In [ ]:
initializer = tf.initializers.RandomNormal(stddev=0.01)
net = tf.keras.Sequential()
net.add(tf.keras.layers.Dense(1, kernel_initializer=initializer))

## defining the loss funtion

In [ ]:
loss = tf.keras.losses.MeanSquaredError()

## defining the optimization algorithim

In [ ]:
trainer = tf.keras.optimizers.SGD(learning_rate=0.03)

## training

In [ ]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        with tf.GradientTape() as tape:
            l = loss(net(X, training=True), y)
        grads = tape.gradient(l, net.trainable_variables)
        trainer.apply_gradients(zip(grads, net.trainable_variables))
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000385
epoch 2, loss 0.000098
epoch 3, loss 0.000098


we compare the model parameters learned by training on finite data and the actual parameters that generated our dataset. To access parameters, we first access the layer that we need from net and then access that layer’s weights and bias. As in our from-scratch implementation, note that our estimated parameters are close to their ground-truth counterparts.

In [ ]:
w = net.get_weights()[0]
print('error in estimating w', true_w - tf.reshape(w, true_w.shape))
b = net.get_weights()[1]
print('error in estimating b', true_b - b)

error in estimating w tf.Tensor([-0.00012422 -0.00023961], shape=(2,), dtype=float32)
error in estimating b [-0.00034237]
